# Группировка и агрегация данных

Импорт библиотек

In [1]:
import polars as pl

Загрузим следующий набор данных:

In [2]:
sourse = "https://raw.githubusercontent.com/m-ardat/Library_Polars/main/dataset/diamonds.csv"
df = pl.read_csv(sourse, columns=["carat", "cut", "color", "clarity", "depth", "table", "price", "x", "y", "z"])

df.head(3)

carat,cut,color,clarity,depth,table,price,x,y,z
f64,str,str,str,f64,f64,i64,f64,f64,f64
0.23,"""Ideal""","""E""","""SI2""",61.5,55.0,326,3.95,3.98,2.43
0.21,"""Premium""","""E""","""SI1""",59.8,61.0,326,3.89,3.84,2.31
0.23,"""Good""","""E""","""VS1""",56.9,65.0,327,4.05,4.07,2.31


Имеем набор данных о бриллиантах. Краткое описание данных:
- **carat**: вес алмаза
- **cut**: качество огранки
- **color**: цвет алмаза
- **clarity**: измерение прозрачности алмаза
- **depth**: глубина
- **table**: ширина верхней части алмаза относительно самой широкой точки
- **price**: цена в долларах США
- **x**: длина в мм
- **y**: ширина в мм
- **z**: глубина в мм

## Группировка данных

В *polars* есть методы, посвященные группировке данных и их агрегации. С помощью этого можно группировать данные по определенным категориям и вычислять агрегированные значения, такие как сумма, среднее или максимальное значение.

### Метод `polars.dataframe.group_by.GroupBy.__iter__`

Метод `polars.dataframe.group_by.GroupBy.__iter__` позволяет итерироваться по группам после операции группировки. Это полезно, когда необходимо обработать каждую группу данных отдельно, например, применить кастомную логику, сохранить результаты или визуализировать подгруппы.

Например, сгруппируем алмазы по качеству огранки (cut), а затем переберём каждую группу:

In [3]:
for name, data in df.group_by(["cut"]):
    print(f"Группа: {name}")
    print(data.head(4)) # Показываем только 4 строчки
    print("-" * 30)

Группа: ('Ideal',)
shape: (4, 10)
┌───────┬───────┬───────┬─────────┬───┬───────┬──────┬──────┬──────┐
│ carat ┆ cut   ┆ color ┆ clarity ┆ … ┆ price ┆ x    ┆ y    ┆ z    │
│ ---   ┆ ---   ┆ ---   ┆ ---     ┆   ┆ ---   ┆ ---  ┆ ---  ┆ ---  │
│ f64   ┆ str   ┆ str   ┆ str     ┆   ┆ i64   ┆ f64  ┆ f64  ┆ f64  │
╞═══════╪═══════╪═══════╪═════════╪═══╪═══════╪══════╪══════╪══════╡
│ 0.23  ┆ Ideal ┆ E     ┆ SI2     ┆ … ┆ 326   ┆ 3.95 ┆ 3.98 ┆ 2.43 │
│ 0.23  ┆ Ideal ┆ J     ┆ VS1     ┆ … ┆ 340   ┆ 3.93 ┆ 3.9  ┆ 2.46 │
│ 0.31  ┆ Ideal ┆ J     ┆ SI2     ┆ … ┆ 344   ┆ 4.35 ┆ 4.37 ┆ 2.71 │
│ 0.3   ┆ Ideal ┆ I     ┆ SI2     ┆ … ┆ 348   ┆ 4.31 ┆ 4.34 ┆ 2.68 │
└───────┴───────┴───────┴─────────┴───┴───────┴──────┴──────┴──────┘
------------------------------
Группа: ('Very Good',)
shape: (4, 10)
┌───────┬───────────┬───────┬─────────┬───┬───────┬──────┬──────┬──────┐
│ carat ┆ cut       ┆ color ┆ clarity ┆ … ┆ price ┆ x    ┆ y    ┆ z    │
│ ---   ┆ ---       ┆ ---   ┆ ---     ┆   ┆ ---   ┆ ---  ┆ --

Метод `GroupBy.__iter__()` возвращает итератор, который позволяет проходить по каждой группе, полученной после `group_by()`. На каждой итерации получаем кортеж из двух элементов:
- `name` — имя группы (или комбинация значений, определяющих группу), представлено как кортеж.
- `data` — *DataFrame*, содержащий строки, принадлежащие этой группе.

### Встроенные методы. Готовые агрегатные функции.

После группировки данных с помощью `group_by()` часто нужно вычислить статистические показатели по каждой группе. *Polars* предоставляет удобные встроенные методы для самых распространённых агрегаций. Рассмотрим следующие методы:
- `max()` - максимальное значение;
- `min()` - минимальное значение;
- `mean()` - среднее арифметическое;
- `median()` - медиана;
- `n_unique()` - количество уникальных значений;
- `quantile()` - квантиль;
- `sum()` - сумма значений;
- `len()` - возвращает количество строк в каждой группе;
- `last()` - возвращает последнее значение;
- `first()` - возвращает первое значение.

In [4]:
df.group_by("cut").max()

cut,carat,color,clarity,depth,table,price,x,y,z
str,f64,str,str,f64,f64,i64,f64,f64,f64
"""Ideal""",3.5,"""J""","""VVS2""",66.7,63.0,18806,9.65,31.8,6.03
"""Fair""",5.01,"""J""","""VVS2""",79.0,95.0,18574,10.74,10.54,6.98
"""Premium""",4.01,"""J""","""VVS2""",63.0,62.0,18823,10.14,58.9,8.06
"""Very Good""",4.0,"""J""","""VVS2""",64.9,66.0,18818,10.01,9.94,31.8
"""Good""",3.01,"""J""","""VVS2""",67.0,66.0,18788,9.44,9.38,5.79


In [5]:
df.group_by("cut").min()

cut,carat,color,clarity,depth,table,price,x,y,z
str,f64,str,str,f64,f64,i64,f64,f64,f64
"""Fair""",0.22,"""D""","""I1""",43.0,49.0,337,0.0,0.0,0.0
"""Ideal""",0.2,"""D""","""I1""",43.0,43.0,326,0.0,0.0,0.0
"""Premium""",0.2,"""D""","""I1""",58.0,51.0,326,0.0,0.0,0.0
"""Good""",0.23,"""D""","""I1""",54.3,51.0,327,0.0,0.0,0.0
"""Very Good""",0.2,"""D""","""I1""",56.8,44.0,336,0.0,0.0,0.0


Для строковых столбцов (color, clarity) максимум/минимум — это лексикографически наибольшее/наименьшее значение.

In [6]:
df.group_by("cut").sum()

cut,carat,color,clarity,depth,table,price,x,y,z
str,f64,str,str,f64,f64,i64,f64,f64,f64
"""Good""",4166.1,null,null,305967.0,287955.9,19275009,28645.08,28703.75,17855.42
"""Very Good""",9742.7,null,null,746888.4,700226.2,48107623,69359.09,69713.45,43009.52
"""Ideal""",15146.84,null,null,1.3299e6,1.2058e6,74513487,118691.07,118963.24,73304.61
"""Premium""",12300.95,null,null,844901.1,810167.4,63221498,82385.88,81985.82,50297.49
"""Fair""",1684.28,null,null,103107.1,95076.6,7017600,10057.5,9954.07,6412.26


In [7]:
df.group_by("cut").mean()

cut,carat,color,clarity,depth,table,price,x,y,z
str,f64,str,str,f64,f64,f64,f64,f64,f64
"""Ideal""",0.702837,null,null,61.709401,55.951668,3457.54197,5.507451,5.52008,3.401448
"""Premium""",0.891955,null,null,61.264673,58.746095,4584.257704,5.973887,5.944879,3.647124
"""Good""",0.849185,null,null,62.365879,58.694639,3928.864452,5.838785,5.850744,3.639507
"""Fair""",1.046137,null,null,64.041677,59.053789,4358.757764,6.246894,6.182652,3.98277
"""Very Good""",0.806381,null,null,61.818275,57.95615,3981.759891,5.740696,5.770026,3.559801


In [8]:
df.group_by("cut").median()

cut,carat,color,clarity,depth,table,price,x,y,z
str,f64,str,str,f64,f64,f64,f64,f64,f64
"""Fair""",1.0,null,null,65.0,58.0,3282.0,6.175,6.1,3.97
"""Ideal""",0.54,null,null,61.8,56.0,1810.0,5.25,5.26,3.23
"""Good""",0.82,null,null,63.4,58.0,3050.5,5.98,5.99,3.7
"""Premium""",0.86,null,null,61.4,59.0,3185.0,6.11,6.06,3.72
"""Very Good""",0.71,null,null,62.1,58.0,2648.0,5.74,5.77,3.56


Для строковых столбцов (color, clarity) в методах `mean()`, `median()`, `sum()` возвращается null.

In [9]:
df.group_by("cut").n_unique()

cut,carat,color,clarity,depth,table,price,x,y,z
str,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""Fair""",185,7,8,180,43,1267,397,402,300
"""Very Good""",231,7,8,81,85,5840,496,495,320
"""Premium""",251,7,8,51,22,6014,527,527,341
"""Good""",199,7,8,106,79,3086,471,473,304
"""Ideal""",232,7,8,74,56,7281,499,495,319


In [10]:
df.group_by("cut").quantile(0.3) # Указываем, какой квантиль хоти отобразить (значения от 0 до 1).

cut,carat,color,clarity,depth,table,price,x,y,z
str,f64,str,str,f64,f64,f64,f64,f64,f64
"""Ideal""",0.38,null,null,61.4,55.0,965.0,4.68,4.7,2.89
"""Fair""",0.73,null,null,64.5,57.0,2301.0,5.77,5.72,3.65
"""Premium""",0.5,null,null,60.7,58.0,1248.0,5.09,5.06,3.1
"""Very Good""",0.5,null,null,61.2,57.0,1185.0,5.04,5.05,3.1
"""Good""",0.51,null,null,62.2,57.0,1388.0,5.12,5.13,3.22


Для строковых столбцов (color, clarity) возвращается null.

In [11]:
df.group_by("cut").len(name="cnt_rows")

cut,cnt_rows
str,u32
"""Fair""",1610
"""Good""",4906
"""Premium""",13791
"""Very Good""",12082
"""Ideal""",21551


In [12]:
df.group_by("cut").first()

cut,carat,color,clarity,depth,table,price,x,y,z
str,f64,str,str,f64,f64,i64,f64,f64,f64
"""Very Good""",0.24,"""J""","""VVS2""",62.8,57.0,336,3.94,3.96,2.48
"""Premium""",0.21,"""E""","""SI1""",59.8,61.0,326,3.89,3.84,2.31
"""Ideal""",0.23,"""E""","""SI2""",61.5,55.0,326,3.95,3.98,2.43
"""Fair""",0.22,"""E""","""VS2""",65.1,61.0,337,3.87,3.78,2.49
"""Good""",0.23,"""E""","""VS1""",56.9,65.0,327,4.05,4.07,2.31


In [13]:
df.group_by("cut").last()

cut,carat,color,clarity,depth,table,price,x,y,z
str,f64,str,str,f64,f64,i64,f64,f64,f64
"""Ideal""",0.75,"""D""","""SI2""",62.2,55.0,2757,5.83,5.87,3.64
"""Very Good""",0.7,"""D""","""SI1""",62.8,60.0,2757,5.66,5.68,3.56
"""Fair""",0.71,"""D""","""VS1""",65.4,59.0,2747,5.62,5.58,3.66
"""Good""",0.72,"""D""","""SI1""",63.1,55.0,2757,5.69,5.75,3.61
"""Premium""",0.86,"""H""","""SI2""",61.0,58.0,2757,6.15,6.12,3.74


###  Метод `polars.dataframe.group_by.GroupBy.all`

Данный метод агрегирует все столбцы, объединяя значения в списки для каждой группы.

In [14]:
df.group_by("cut").all()

cut,carat,color,clarity,depth,table,price,x,y,z
str,list[f64],list[str],list[str],list[f64],list[f64],list[i64],list[f64],list[f64],list[f64]
"""Ideal""","[0.23, 0.23, … 0.75]","[""E"", ""J"", … ""D""]","[""SI2"", ""VS1"", … ""SI2""]","[61.5, 62.8, … 62.2]","[55.0, 56.0, … 55.0]","[326, 340, … 2757]","[3.95, 3.93, … 5.83]","[3.98, 3.9, … 5.87]","[2.43, 2.46, … 3.64]"
"""Very Good""","[0.24, 0.24, … 0.7]","[""J"", ""I"", … ""D""]","[""VVS2"", ""VVS1"", … ""SI1""]","[62.8, 62.3, … 62.8]","[57.0, 57.0, … 60.0]","[336, 336, … 2757]","[3.94, 3.95, … 5.66]","[3.96, 3.98, … 5.68]","[2.48, 2.47, … 3.56]"
"""Fair""","[0.22, 0.86, … 0.71]","[""E"", ""E"", … ""D""]","[""VS2"", ""SI2"", … ""VS1""]","[65.1, 55.1, … 65.4]","[61.0, 69.0, … 59.0]","[337, 2757, … 2747]","[3.87, 6.45, … 5.62]","[3.78, 6.33, … 5.58]","[2.49, 3.52, … 3.66]"
"""Good""","[0.23, 0.31, … 0.72]","[""E"", ""J"", … ""D""]","[""VS1"", ""SI2"", … ""SI1""]","[56.9, 63.3, … 63.1]","[65.0, 58.0, … 55.0]","[327, 335, … 2757]","[4.05, 4.34, … 5.69]","[4.07, 4.35, … 5.75]","[2.31, 2.75, … 3.61]"
"""Premium""","[0.21, 0.29, … 0.86]","[""E"", ""I"", … ""H""]","[""SI1"", ""VS2"", … ""SI2""]","[59.8, 62.4, … 61.0]","[61.0, 58.0, … 58.0]","[326, 334, … 2757]","[3.89, 4.2, … 6.15]","[3.84, 4.23, … 6.12]","[2.31, 2.63, … 3.74]"


## Агрегация данных

### Метод `polars.dataframe.group_by.GroupBy.agg`

Метод `GroupBy.agg()` — основной способ агрегирования данных в *polars*. Он позволяет вычислить суммы, средние, минимумы, максимумы и другие метрики для каждой группы после операции `group_by`, т.е. вычисляет агрегации для каждой группы в результате операции группировки.

Параметры метода:
- `aggs` - Агрегации, вычисляемые для каждой группы, передаваемые как позиционные аргументы.
- `named_aggs` - Дополнительные агрегации, передаваемые как именованные аргументы. Имена параметров используются как названия результирующих столбцов.

Возвращает *DataFrame*, где каждая строка соответствует одной группе, а столбцы содержат результаты агрегаций.

Например, сгруппируем алмазы по качеству огранки (cut), и вычислим агрегат для столбца вес алмаза:

In [15]:
df.group_by("cut").agg(pl.col("carat"))

cut,carat
str,list[f64]
"""Very Good""","[0.24, 0.24, … 0.7]"
"""Ideal""","[0.23, 0.23, … 0.75]"
"""Premium""","[0.21, 0.29, … 0.86]"
"""Fair""","[0.22, 0.86, … 0.71]"
"""Good""","[0.23, 0.31, … 0.72]"


- `df.group_by("cut")`: группируем строки *DataFrame* уникальным значениям в столбце "cut".
- `agg(pl.col("carat"))`: выбираем значения из столбца "carat" для каждой группы.
В результате выполнения кода получен новый *DataFrame*, где для каждой категории в столбце "cut" будет список значений из столбца "carat", т.е. метод позволил увидеть, какие значения соответсвуют каждой категории.

Можно вычислить несколько агрегаций одновременно, передав список выражений.

In [16]:
df.group_by("cut").agg(pl.col("carat").sum(), pl.col("price").mean())

cut,carat,price
str,f64,f64
"""Good""",4166.1,3928.864452
"""Very Good""",9742.7,3981.759891
"""Ideal""",15146.84,3457.54197
"""Premium""",12300.95,4584.257704
"""Fair""",1684.28,4358.757764


Или вот так:

In [17]:
df.group_by("cut").agg(pl.sum("carat").name.suffix("_sum"), pl.mean("price").name.suffix("_mean"))

cut,carat_sum,price_mean
str,f64,f64
"""Ideal""",15146.84,3457.54197
"""Fair""",1684.28,4358.757764
"""Very Good""",9742.7,3981.759891
"""Premium""",12300.95,4584.257704
"""Good""",4166.1,3928.864452


Конструкция `name.suffix("_mean")` добавляет суффикс к колонкам.

Или же использовать именованные аргументы, чтобы легко задать имя столбцам:

In [18]:
df.group_by("cut").agg(mean_carat = pl.mean("carat"))

cut,mean_carat
str,f64
"""Ideal""",0.702837
"""Premium""",0.891955
"""Fair""",1.046137
"""Good""",0.849185
"""Very Good""",0.806381
